# Houston 311 Service Requests Data Mining Project

## Group members
Stephen Huang

Levi Villarreal

Joshua Wong

Andrew Young


### Code Imports

In [50]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
import time
import datetime
# TODO: DROP THE STATUS COLUMN

### Import data

In [2]:

# Data is from http://www.houstontx.gov/311/
names = ["CASE NUMBER", "SR LOCATION", "COUNTY", "DISTRICT", "NEIGHBORHOOD", "TAX ID", 
         "TRASH QUAD", "RECYCLE QUAD", "TRASH DAY", "HEAVY TRASH DAY", "RECYCLE DAY", 
         "KEY MAP", "MANAGEMENT DISTRICT", "DEPARTMENT", "DIVISION", "SR TYPE", "QUEUE", 
         "SLA", "STATUS", "SR CREATE DATE", "DUE DATE", "DATE CLOSED", "OVERDUE", "TITLE", 
         "x", "y", "LATITUDE", "LONGITUDE", "CHANNEL TYPE"]

data = pd.read_csv('311-Public-Data-Extract-2018-clean.txt', sep="|", header=None, names=names)
print(data.shape)
data.head()

FileNotFoundError: [Errno 2] File b'insert filename' does not exist: b'insert filename'

## Dimensionality Reduction
Right off the bat, we can see that there are many features that do not matter at all to our project.
We can go ahead and get rid of these to help prevent against the curse of dimensionality.

In [ ]:
data = data.drop(columns=['SR LOCATION', 'CASE NUMBER', 'TRASH QUAD', 'RECYCLE QUAD', 'TRASH DAY', 'HEAVY TRASH DAY', 'RECYCLE DAY', 'TAX ID', 'KEY MAP', 'DUE DATE', 'DATE CLOSED', 'TITLE', 'x', 'y', 'LONGITUDE', 'LATITUDE'])
print(data.shape)

data.head()

### Reasoning behind dropping the attributes we did

Case number - This was dropped because the case number is a unique identifier assigned to every 311 request, and thus could not be used to classify anything.

Trash Quad - There are requests that are not related to trash collection. Those requests might be misclassified because they are in a different trash quad as other requests in the dataset.

Recycle Quad - Same as Trash Quad

Trash Day - Same as Trash Quad

Heavy Trash Day - Same as Trash Quad

Recycle day - Same as Trash Quad
 
SR Location - Same reasoning as case number.
 
Tax ID - This ID is unique to the property that requested it.
 
Key Map - This attribute is used in the cities internal system to keep track of requests and does not have any actual relation to the request
 
Due Date - We are interested in predicting overdueness, and that it its own seperate column.
 
Date Closed - Same reasoning as due date.
 
Title - This is a another unique identifier assigned to every 311 request, and thus could not be used to classify anything.
 
x - We already have latitude, and this attribute an approximation of latitude.
 
y - We already have longitude, and this attribute an approximation of longitude.

latitude - This is too speficific for our needs, and we have other location metrics.
 
longitude - same as latitude

## Data Cleaning

Drop all service requests that are not closed

In [ ]:
oldNumRows = data.shape[0]
data = data[data.STATUS == 'Closed']

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Normalize all counties and drop rows where county is unknown

In [ ]:
print('Counties:')
print(data['COUNTY'].value_counts())
print()

oldNumRows = data.shape[0]
data['COUNTY'] = data['COUNTY'].replace(['HARRIS', 'FORT BEND', 'MONTGOMERY'], ['Harris County', 'Fort Bend County', 'Montgomery County'])
data = data[data.COUNTY != 'Unknown']

print('Counties:')
print(data['COUNTY'].value_counts())
print()
print("Number of rows with COUNTY == 'Unknown' dropped: ", oldNumRows - data.shape[0])

Drop rows where district is unknown

In [ ]:
oldNumRows = data.shape[0]
data = data[data.DISTRICT != 'Unknown']

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Normalize all districts

In [ ]:
data['MANAGEMENT DISTRICT'] = data['MANAGEMENT DISTRICT'].replace(['HCID #3 TRACT 19 (Upper Kirby)', 'HCID #3 TRACT 47 (Upper Kirby)', 'East End MD', 'Greater Northside', 'Sharpstown'], ['HCID #3 (Upper Kirby)', 'HCID #3 (Upper Kirby)', 'East End', 'Greater Northside MD', 'Sharpstown MD'])
print('Management Districts:')
print(data['MANAGEMENT DISTRICT'].value_counts().sort_index())

Drop all rows with unknown channel types

In [ ]:
oldNumRows = data.shape[0]
data = data[data['CHANNEL TYPE'] != 'Unknown']

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Drop all rows with null values

In [ ]:
oldNumRows = data.shape[0]
data = data.dropna()

print("Number of rows dropped: ", oldNumRows - data.shape[0])

## Feature Transformation

Currently, the request date would be draw conclusions from, so we transformed the feature to instead be the month in which the service request was created, to account seasonal differences in service request time.

In [ ]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
data['SR CREATE DATE'] = data['SR CREATE DATE'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
data['SR CREATE DATE'] = data['SR CREATE DATE'].apply(lambda x : months[x.month-1])

data = data.rename(columns={'SR CREATE DATE': 'SR MONTH'})
data.head()

Currently, the overdue is continuous, which makes it hard to use in classification algorithms. To combat this, we want to bin the values, so that there are only a few possible values.

The values we want bin into are on time tasks (negative values), tasks done within a week past due date (0 < x < 7), tasks done within a month past due date (7 <= x < 30), tasks done over a month past due date (x >= 30).

In [ ]:
def bin_overdue (val):
    val = float(val)
    
    if val < 0:
        return "On Time"
    elif val < 7:
        return "Week"
    elif val < 30:
        return "Month"
    else:
        return "More"
    
data["OVERDUE"] = data["OVERDUE"].apply(bin_overdue)
    
print(data['OVERDUE'].value_counts().sort_index())
data.head()

## Export data to a CSV

In [ ]:
data.to_csv('houston-311-sanitized.csv', index=False)

## Read in Sanitized Data

In [69]:
data = pd.read_csv('houston-311-sanitized.csv', sep=",", header='infer')
#data = data.drop(columns='Unnamed: 0')
print(data.shape)
data.head()

(152212, 13)


,COUNTY,DISTRICT,NEIGHBORHOOD,MANAGEMENT DISTRICT,DEPARTMENT,DIVISION,SR TYPE,QUEUE,SLA,STATUS,SR MONTH,OVERDUE,CHANNEL TYPE
0,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1.0,Closed,January,On Time,Voice In
1,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1.0,Closed,January,On Time,Voice In
2,Harris County,I,GREATER HOBBY AREA,HCID #9,PWE Public Works Engineering,Street and Drainage,Street Hazard,ROWM_StreetMain,10.0,Closed,January,On Time,Voice In
3,Harris County,D,SUNNYSIDE,Five Corners HCID #10B,NS Neighborhood Services,Investigations,Nuisance On Property,NS_Dispatch,180.0,Closed,January,More,Voice In
4,Harris County,A,SPRING BRANCH NORTH,Spring Branch MD,SWM Solid Waste Management,General Support Services,Dumpster Complaint,SWM_Complaints,21.0,Closed,January,On Time,Voice In


In [75]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# TODO move dropping status up to preprocesssing before export
data = data.drop(["STATUS"], axis=1)
num_data = data.copy()
cat_columns = ["COUNTY", "DISTRICT", "NEIGHBORHOOD", "MANAGEMENT DISTRICT", "DEPARTMENT", "DIVISION",
              "SR TYPE", "QUEUE", "SR MONTH", "CHANNEL TYPE"]

for c in cat_columns:
    prev_column = num_data[c]
    encoder = preprocessing.LabelEncoder()
    num_data[c] = encoder.fit_transform(prev_column)

print(num_data.head())
train, test = train_test_split(num_data, train_size = 0.05, test_size = 0.05)

KeyError: "['STATUS'] not found in axis"

In [71]:
down_data = train

labels = down_data['OVERDUE']
df_features = down_data.drop(['OVERDUE'], axis=1)
print(df_labels.value_counts().sort_index())

Month       142
More         92
On Time    1121
Week        167
Name: OVERDUE, dtype: int64


## Decision Trees

In [72]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score

features_train, features_test, labels_train, labels_test = train_test_split(df_features, labels, test_size = 0.2)
print('Training set size:',len(labels_train))
print('Test set size:    ',len(labels_test))
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
predict_labels_test = clf.predict(features_test)
print('Accuracy of decision tree classifier:',accuracy_score(labels_test, predict_labels_test))

Training set size: 120551
Test set size:     30138
Accuracy of decision tree classifier: 0.7822682327958059


## Support Vector Machines (SVM)

In [53]:
pipe = Pipeline(steps=[('scaler',StandardScaler()),('reduce_dim',PCA()),('clf',SVC(gamma='scale'))])
param_grid = {
    'reduce_dim__n_components': list(range(5, 11)),
    'clf__kernel':('linear','rbf','poly')
}
accuracies = cross_val_score(GridSearchCV(pipe,param_grid,cv=5,scoring='accuracy').fit(df_features,labels), df_features, labels, cv=10)
print('Official accuracy:',np.mean(accuracies))

Official accuracy: 0.71684520075022


## Neural Networks(NN)

In [55]:
from sklearn.neural_network import MLPClassifier
pipe = Pipeline(steps=[('scaler',StandardScaler()),('clf',MLPClassifier())])
param_grid = {
    'clf__hidden_layer_sizes':((30,),(40,),(50,),(60,)),
    'clf__activation':('logistic','tanh','relu')
}
accuracies = cross_val_score(GridSearchCV(pipe,param_grid,cv=5,scoring='accuracy').fit(df_features,labels), df_features, labels, cv=5)
print('Official accuracy:',np.mean(accuracies))

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

...

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

Official accuracy: 0.718802055251985


/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## k-Nearest Neighbor (KNN)

In [73]:
from sklearn.neighbors import KNeighborsClassifier
pipe = Pipeline(steps=[('scaler',StandardScaler()),('reduce_dim',PCA()),('clf',KNeighborsClassifier(n_neighbors=7))])
accuracies = cross_val_score(pipe,df_features,labels,cv=5)
print('Average accuracy:',np.mean(accuracies))

Average accuracy: 0.7799972413401078


## Naive Bayes
Naive Bayes is not a good option for this dataset due to correlation between attributes. For example, requests from the same neighborhood would be from the same county.

In [74]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
accuracies = cross_val_score(clf,df_features,labels,cv=10)
print('Average accuracy:',np.mean(accuracies))

Average accuracy: 0.5669426919342034


## AdaBoost

In [59]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=150)
accuracies = cross_val_score(clf, df_features, labels, cv=5)
print('Average accuracy:',np.mean(accuracies))

Average accuracy: 0.6148608101628366


## Random Forest

In [61]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
param_grid = {
    'max_depth':list(range(35,55)),
    'min_samples_leaf':(8,10,12),
    'max_features':('sqrt','log2')
}
accuracies = cross_val_score(GridSearchCV(clf,param_grid,cv=5,scoring='accuracy').fit(df_features,labels), df_features, labels, cv=5)
print('Official accuracy:',np.mean(accuracies))

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/si

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/si

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/si

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/si

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/si

...

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/si

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/si

/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/si

Official accuracy: 0.735239250198593


/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Final Model